# Example: Flutter calculation

This notebook explains the process of model generation, simulation running and output analysis for an aeroelastic flutter simulation using SHARPy. It uses the model-generator to produce a single model.

In [1]:
import numpy as np
import os
import pdb
import importlib
import cases.models_generator.gen_main as gm
import sharpy.utils.algebra as algebra
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D

model_route = os.getcwd()+'/example_flutter'

print(model_route)

/home/pablodfs/FYP/Projects-SHARPy/aeroelasticPMOR_Optimization/parametric_aircraft/example_flutter


## Model definition


In [2]:
# First some inputs
wing_semispan = 12 
bound_panels = 4 # This controls the chordwise discretisation of the aerogrid
components=['fuselage', 'wing_r', 'winglet_r',
                              'wing_l', 'winglet_l', 'vertical_tail',
                              'horizontal_tail_right', 'horizontal_tail_left']
# aeroelasticity parameters
main_ea = 0.3  # Wing elastic axis from LE as %
main_cg = 0.3  # Not sure about this input
sigma = 1.5
c_ref = 1.0

#########
# wings #
#########
#
ea = 1e7
ga = 1e5
gj = 1e4
eiy = 2e4
eiz = 4e6
m_bar_main = 0.75
j_bar_main = 0.075
mass_main1 = np.diag([m_bar_main, m_bar_main, m_bar_main,
                      j_bar_main, 0.5 * j_bar_main, 0.5 * j_bar_main])
stiffness_main1 = sigma * np.diag([ea, ga, ga, gj, eiy, eiz])
stiffness_main = np.zeros((1, 6, 6))
stiffness_main[0] = stiffness_main1
mass_main = np.zeros((1, 6, 6))
mass_main[0] = mass_main1
############
# fuselage #
############
#
sigma_fuselage = 10
m_bar_fuselage = 0.2
j_bar_fuselage = 0.08
stiffness_fuselage1 = np.diag([ea, ga, ga, gj, eiy, eiz]) * sigma * sigma_fuselage
stiffness_fuselage1[4, 4] = stiffness_fuselage1[5, 5]
mass_fuselage1 = np.diag([m_bar_fuselage,
                          m_bar_fuselage,
                          m_bar_fuselage,
                          j_bar_fuselage,
                          j_bar_fuselage * 0.5,
                          j_bar_fuselage * 0.5])
stiffness_fuselage = np.zeros((1, 6, 6))
stiffness_fuselage[0] = stiffness_fuselage1
mass_fuselage = np.zeros((1, 6, 6))
mass_fuselage[0] = mass_fuselage1
########
# tail #
########
#
sigma_tail = 100
m_bar_tail = 0.3
j_bar_tail = 0.08
stiffness_tail1 = np.diag([ea, ga, ga, gj, eiy, eiz]) * sigma * sigma_tail
stiffness_tail1[4, 4] = stiffness_tail1[5, 5]
mass_tail1 = np.diag([m_bar_tail,
                      m_bar_tail,
                      m_bar_tail,
                      j_bar_tail,
                      j_bar_tail * 0.5,
                      j_bar_tail * 0.5])
stiffness_tail = np.zeros((1, 6, 6))
stiffness_tail[0] = stiffness_tail1
mass_tail = np.zeros((1, 6, 6))
mass_tail[0] = mass_tail1

######################################
# Lumped mass at fuselage/wing cross #
######################################
n_lumped_mass = 1  # Number of lumped masses
lumped_mass_nodes = np.zeros((n_lumped_mass,), dtype=int)  # Maps lumped mass to nodes
lumped_mass = np.zeros((n_lumped_mass,))  # Array of lumped masses in kg
lumped_mass[0] = 50
lumped_mass_inertia = np.zeros((n_lumped_mass, 3, 3))  # 3x3 inertia to the previous masses
lumped_mass_position = np.zeros((n_lumped_mass, 3))  # Relative position to the belonging node in B FoR

##############
# Components #
##############
g1c = dict()
g1c['fuselage'] = {'workflow': ['create_structure', 'create_aero0'],
                   'geometry': {'length': 10,
                                'num_node': 9,
                                'direction': [1., 0., 0.],
                                'sweep': 0.,
                                'dihedral': 0.},
                   'fem': {'stiffness_db': stiffness_fuselage,
                           'mass_db': mass_fuselage,
                           'frame_of_reference_delta': [0, 1., 0.],
                           'lumped_mass': lumped_mass,
                           'lumped_mass_nodes': lumped_mass_nodes,
                           'lumped_mass_inertia': lumped_mass_inertia,
                           'lumped_mass_position': lumped_mass_position}
                   }

g1c['wing_r'] = {'workflow': ['create_structure', 'create_aero'],
                 'geometry': {'length': wing_semispan,
                              'num_node': 13,
                              'direction': [0., 1., 0.],
                              'sweep': 0. * np.pi / 180,
                              'dihedral': 0.},
                 'fem': {'stiffness_db': stiffness_main,
                         'mass_db': mass_main,
                         'frame_of_reference_delta': [-1, 0., 0.]},
                 'aero': {'chord': [1., 1.],
                          'elastic_axis': main_ea,
                          'surface_m': bound_panels}
                 }
g1c['winglet_r'] = {'workflow': ['create_structure', 'create_aero'],
                    'geometry': {'length': 4,
                                 'num_node': 5,
                                 'direction': [0., 1., 0.],
                                 'sweep': 0. * np.pi / 180,
                                 'dihedral': 20. * np.pi / 180},
                    'fem': {'stiffness_db': stiffness_main,
                            'mass_db': mass_main,
                            'frame_of_reference_delta': [-1, 0., 0.]},
                    'aero': {'chord': [1., 1.],
                             'elastic_axis': main_ea,
                             'surface_m': bound_panels,
                             'merge_surface': True}
                    }
g1c['wing_l'] = {'symmetric': {'component': 'wing_r'}}
g1c['winglet_l'] = {'symmetric': {'component': 'winglet_r'}}
g1c['vertical_tail'] = {'workflow': ['create_structure', 'create_aero'],
                        'geometry': {'length': 2.5,
                                     'num_node': 9,
                                     'direction': [0., 0., 1.],
                                     'sweep': None,
                                     'dihedral': None},
                        'fem': {'stiffness_db': stiffness_tail,
                                'mass_db': mass_tail,
                                'frame_of_reference_delta': [-1., 0., 0.]},
                        'aero': {'chord': [0.45, 0.45],
                                 'elastic_axis': 0.5,
                                 'surface_m': bound_panels}
                        }
g1c['horizontal_tail_right'] = {'workflow': ['create_structure', 'create_aero'],
                                'geometry': {'length': 2.5,
                                             'num_node': 9,
                                             'direction': [0., 1., 0.],
                                             'sweep': 0.,
                                             'dihedral': 0.},
                                'fem': {'stiffness_db': stiffness_tail,
                                        'mass_db': mass_tail,
                                        'frame_of_reference_delta': [-1, 0., 0.]},
                                'aero': {'chord': [0.5, 0.5],
                                         'elastic_axis': 0.5,
                                         'surface_m': bound_panels}
                                }
g1c['horizontal_tail_left'] = {'symmetric': {'component': 'horizontal_tail_right'}}

g1c_output = {i: g1c[i] for i in components}
    

## Model generation

In [8]:
# First some inputs
model_name = 'example_flutter'
g1mm = {'model_name': model_name,
        'model_route': model_route,
         'iterate_type': 'Full_Factorial',
         'write_iterate_vars': True,
         'iterate_vars': {'fuselage*geometry-length': np.array([10]),
                          'wing_r*geometry-length': np.array([8]),
                          'winglet_r*geometry-dihedral': np.pi / 180 * np.array([20])},
         'iterate_labels': {'label_type': 'number',
                            'print_name_var': 0},
        'assembly': {'include_aero': 1,
                     'default_settings': 1,  # beam_number and aero surface and
                     # surface_distribution
                     # selected by default one
                     # per component
                     'fuselage': {'upstream_component': '',
                                  'node_in_upstream': 0},
                     'wing_r': {'keep_aero_node': 1,
                                'upstream_component': 'fuselage',
                                'node_in_upstream': 0},
                     'winglet_r': {'keep_aero_node': 1,
                                   'upstream_component': 'wing_r',
                                   'node_in_upstream': -1},
                     'wing_l': {'upstream_component': 'fuselage',
                                'node_in_upstream': 0},
                     'winglet_l': {'upstream_component': 'wing_l',
                                   'node_in_upstream': -1},
                     'vertical_tail': {'upstream_component': 'fuselage',
                                       'node_in_upstream': -1},
                     'horizontal_tail_right': {'upstream_component': 'vertical_tail',
                                               'node_in_upstream': -1},
                     'horizontal_tail_left': {'upstream_component': 'vertical_tail',
                                              'node_in_upstream': -1}
                     }
        }
for ki in ['fuselage', 'wing_r', 'winglet_r',
           'wing_l', 'winglet_l', 'vertical_tail',
           'horizontal_tail_right', 'horizontal_tail_left']:

    if (ki not in ['include_aero', 'default_settings'] and
            ki not in components):
        del g1mm['assembly'][ki]

## Simulation definition

In the below simulation we are defining the panels_wake as $8*10$ = bound_panels*10 but we are also defining a Horseshoe vortex, sooo, not sure what we are doing really

Why do we need a u_inf input? To get dt
Why do we need dt?

In [9]:
#####################################################################
# Run a flutter solution around an arbitrary aeroelastic equilibrium#
####################################################################
u_inf = 20
rho = 1.2
c_ref = 1.0
AoA = 0. * np.pi / 180
bound_panels = 8
sol_152 = {'sharpy': {'simulation_input': None,
                      'default_module': 'sharpy.routines.flutter',
                      'default_solution': 'sol_152',
                      'default_solution_vars': {
                          'flutter_reference': 21.,
                          'root_method': 'secant',
                          'velocity_increment': 10.,
                          'flutter_error': 0.001,
                          'damping_tolerance': 5e-3,
                          'inout_coordinates': 'modes',
                          'secant_max_calls': 15,
                          'rho': rho,
                          'gravity_on': False,
                          'u_inf': u_inf,
                          'panels_wake': bound_panels * 10,
                          'dt': c_ref / bound_panels / u_inf,
                          'c_ref': c_ref,
                          'rom_method': '',
                          'rotationA': [0., AoA, 0.],
                          'horseshoe': True,
                          'num_modes': 20,
                          'fsi_maxiter': 100,
                          'fsi_tolerance': 1e-5,
                          'fsi_relaxation': 0.3,
                          'fsi_load_steps': 1,
                          's_maxiter': 100,
                          's_tolerance': 1e-5,
                          's_relaxation': 1e-3,
                          's_load_steps': 1,
                          's_delta_curved': 1e-4,
                          'add2_flow': [['StaticCoupled', 'plot']],
                      },
                      'default_sharpy': {},
                      'model_route': None}}
#############################################

## Running the simulation

In [10]:
g1 = gm.Model('sharpy', ['sharpy'],
                  model_dict=g1mm,
                  components_dict=g1c_output,
                  simulation_dict=sol_152)
data = g1.run()

No variable airfoil_efficiency defined in component fuselage
No variable airfoil_efficiency defined in component wing_r
No variable airfoil_efficiency defined in component winglet_r
No variable airfoil_efficiency defined in component wing_l
No variable airfoil_efficiency defined in component winglet_l
No variable airfoil_efficiency defined in component vertical_tail
No variable airfoil_efficiency defined in component horizontal_tail_right
No variable airfoil_efficiency defined in component horizontal_tail_left
--------------------------------------------------------------------------------
            ######  ##     ##    ###    ########  ########  ##    ##
           ##    ## ##     ##   ## ##   ##     ## ##     ##  ##  ##
           ##       ##     ##  ##   ##  ##     ## ##     ##   ####
            ######  ######### ##     ## ########  ########     ##
                 ## ##     ## ######### ##   ##   ##           ##
           ##    ## ##     ## ##     ## ##    ##  ##           ##
 

/home/pablodfs/sharpy/sharpy/utils/algebra.py:201: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[1, 2] = -vector[0]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:202: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[2, 0] = -vector[1]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:203: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[0, 1] = -vector[2]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:204: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[2, 1] = vector[0]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:205: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[0, 2] = vector[1]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:206: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[1, 0] = vector[2]
/home/pablodfs/sharpy/sharpy/structure/utils/modalutils.py:178: ComplexWarning:

|      0       |   0.000000   |   2.759347   |   0.439164   |   0.439164   |  -0.000000   |   2.277055   |
|      1       |   0.000000   |   2.759347   |   0.439164   |   0.439164   |  -0.000000   |   2.277055   |
|      2       |   0.000000   |  16.881831   |   2.686827   |   2.686827   |  -0.000000   |   0.372186   |
|      3       |   0.000000   |  16.881831   |   2.686827   |   2.686827   |  -0.000000   |   0.372186   |
|      4       |   0.000000   |  19.695400   |   3.134620   |   3.134620   |  -0.000000   |   0.319018   |
|      5       |  -0.000000   |  19.695400   |   3.134620   |   3.134620   |   0.000000   |   0.319018   |
|      6       |   0.000000   |  31.713913   |   5.047426   |   5.047426   |  -0.000000   |   0.198121   |
|      7       |   0.000000   |  41.368099   |   6.583938   |   6.583938   |  -0.000000   |   0.151885   |
|      8       |   0.000000   |  41.368099   |   6.583938   |   6.583938   |  -0.000000   |   0.151885   |
|      9       |   0.000000   |  45.5

/home/pablodfs/anaconda3/envs/sharpy_env/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


	state-space model produced in form:
			h_{n+1} = A h_{n} + B u_{n}
			with:
	x_n = h_n + Bp u_n
			...done in 3.94 sec
Scaling UVLM system with reference time 0.050000s
Non-dimensional time step set (0.125000)
System scaled in 1.525666s
Generating an instance of LinearBeam
Warning, projecting system with damping onto undamped modes
Scaling beam according to reduced time...
	Setting the beam time step to (0.1250)
Updating C and K matrices and natural frequencies with new normalised time...


/home/pablodfs/sharpy/sharpy/linear/assembler/linearaeroelastic.py:238: ComplexWarning: Casting complex values to real discards the imaginary part
  in_mode_matrix[:2*beam.sys.num_dof, :2*beam.sys.num_modes] = sclalg.block_diag(phi, phi)


Aeroelastic system assembled:
	Aerodynamic states: 715
	Structural states: 40
	Total states: 755
	Inputs: 60
	Outputs: 60
Final system is:
State-space system
States: 755
Inputs: 60
Outputs: 60

Generating an instance of DynamicLoads
Variable print_info has no assigned value in the settings file.
    will default to the value: True
Variable secant_max_calls has no assigned value in the settings file.
    will default to the value: 0
Variable mach_number has no assigned value in the settings file.
    will default to the value: 0.0
Variable flutter_bound has no assigned value in the settings file.
    will default to the value: 0.0
Variable calculate_rootloads has no assigned value in the settings file.
    will default to the value: False
Variable flight_conditions has no assigned value in the settings file.
    will default to the value: {}
Variable gust_regulation has no assigned value in the settings file.
    will default to the value: Continuous_gust
Variable white_noise_covariance

## Output processing

In [6]:
print(type(data[0]))

TypeError: 'NoneType' object is not subscriptable

Testing another approach

In [13]:
cd ./example_flutter/example_flutter0_0_0

/home/pablodfs/FYP/Projects-SHARPy/aeroelasticPMOR_Optimization/parametric_aircraft/example_flutter/example_flutter0_0_0


In [15]:
import sharpy.utils.generate_cases as gc
import sharpy.utils.h5utils as h5utils
import sharpy.utils.solver_interface as solver_interface
import sharpy.sharpy_main

In [16]:
dataX = sharpy.sharpy_main.main(['', 'example_flutter.sharpy'])

--------------------------------------------------------------------------------
            ######  ##     ##    ###    ########  ########  ##    ##
           ##    ## ##     ##   ## ##   ##     ## ##     ##  ##  ##
           ##       ##     ##  ##   ##  ##     ## ##     ##   ####
            ######  ######### ##     ## ########  ########     ##
                 ## ##     ## ######### ##   ##   ##           ##
           ##    ## ##     ## ##     ## ##    ##  ##           ##
            ######  ##     ## ##     ## ##     ## ##           ##
--------------------------------------------------------------------------------
Aeroelastics Lab, Aeronautics Department.
    Copyright (c), Imperial College London.
    All rights reserved.
    License available at https://github.com/imperialcollegelondon/sharpy
Running SHARPy from /home/pablodfs/FYP/Projects-SHARPy/aeroelasticPMOR_Optimization/parametric_aircraft
SHARPy being run is in /home/pablodfs/sharpy
The branch being run is dev_rhea
The 

/home/pablodfs/sharpy/sharpy/utils/algebra.py:201: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[1, 2] = -vector[0]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:202: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[2, 0] = -vector[1]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:203: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[0, 1] = -vector[2]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:204: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[2, 1] = vector[0]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:205: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[0, 2] = vector[1]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:206: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[1, 0] = vector[2]
/home/pablodfs/sharpy/sharpy/structure/utils/modalutils.py:178: ComplexWarning:

|      0       |   0.000000   |   2.759347   |   0.439164   |   0.439164   |  -0.000000   |   2.277055   |
|      1       |   0.000000   |   2.759347   |   0.439164   |   0.439164   |  -0.000000   |   2.277055   |
|      2       |   0.000000   |  16.881831   |   2.686827   |   2.686827   |  -0.000000   |   0.372186   |
|      3       |   0.000000   |  16.881831   |   2.686827   |   2.686827   |  -0.000000   |   0.372186   |
|      4       |   0.000000   |  19.695400   |   3.134620   |   3.134620   |  -0.000000   |   0.319018   |
|      5       |  -0.000000   |  19.695400   |   3.134620   |   3.134620   |   0.000000   |   0.319018   |
|      6       |   0.000000   |  31.713913   |   5.047426   |   5.047426   |  -0.000000   |   0.198121   |
|      7       |   0.000000   |  41.368099   |   6.583938   |   6.583938   |  -0.000000   |   0.151885   |
|      8       |   0.000000   |  41.368099   |   6.583938   |   6.583938   |  -0.000000   |   0.151885   |
|      9       |   0.000000   |  45.5

/home/pablodfs/anaconda3/envs/sharpy_env/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


	state-space model produced in form:
			h_{n+1} = A h_{n} + B u_{n}
			with:
	x_n = h_n + Bp u_n
			...done in 4.52 sec
Scaling UVLM system with reference time 0.050000s
Non-dimensional time step set (0.125000)
System scaled in 1.704875s
Generating an instance of LinearBeam
Warning, projecting system with damping onto undamped modes
Scaling beam according to reduced time...
	Setting the beam time step to (0.1250)
Updating C and K matrices and natural frequencies with new normalised time...


/home/pablodfs/sharpy/sharpy/linear/assembler/linearaeroelastic.py:238: ComplexWarning: Casting complex values to real discards the imaginary part
  in_mode_matrix[:2*beam.sys.num_dof, :2*beam.sys.num_modes] = sclalg.block_diag(phi, phi)


Aeroelastic system assembled:
	Aerodynamic states: 715
	Structural states: 40
	Total states: 755
	Inputs: 60
	Outputs: 60
Final system is:
State-space system
States: 755
Inputs: 60
Outputs: 60

Generating an instance of DynamicLoads
Variable print_info has no assigned value in the settings file.
    will default to the value: True
Variable secant_max_calls has no assigned value in the settings file.
    will default to the value: 0
Variable mach_number has no assigned value in the settings file.
    will default to the value: 0.0
Variable flutter_bound has no assigned value in the settings file.
    will default to the value: 0.0
Variable calculate_rootloads has no assigned value in the settings file.
    will default to the value: False
Variable flight_conditions has no assigned value in the settings file.
    will default to the value: {}
Variable gust_regulation has no assigned value in the settings file.
    will default to the value: Continuous_gust
Variable white_noise_covariance

In [17]:
print(type(dataX))
dataX.__dict__

<class 'sharpy.presharpy.presharpy.PreSharpy'>


{'_settings': True,
 'ts': 0,
 'settings': ConfigObj({'BeamLoader': {'for_pos': array([0., 0., 0.]), 'orientation': array([1., 0., 0., 0.]), 'unsteady': False}, 'AerogridLoader': {'freestream_dir': array([1., 0., 0.]), 'unsteady': True, 'mstar': 1, 'aligned_grid': True, 'control_surface_deflection': [], 'wake_shape_generator': 'StraightWake', 'shift_panels': True, 'control_surface_deflection_generator_settings': {}, 'wake_shape_generator_input': {'u_inf': 20.0, 'u_inf_direction': array([1., 0., 0.]), 'dt': 0.00625, 'dx1': -1.0, 'ndx1': 1, 'r': 1.0, 'dxmax': -1.0}}, 'StaticCoupled': {'n_load_steps': 1, 'max_iter': 100, 'tolerance': 1e-05, 'relaxation_factor': 0.3, 'aero_solver': 'StaticUvlm', 'structural_solver': 'NonLinearStatic', 'print_info': True, 'correct_forces_method': '', 'aero_solver_settings': {'print_info': True, 'horseshoe': True, 'num_cores': 1, 'n_rollup': 1, 'rollup_dt': 0.00625, 'rollup_aic_refresh': 1, 'rollup_tolerance': 0.0001, 'iterative_solver': False, 'iterative_to

In [24]:
print(type(dataX.structure))
print(type(dataX.aero))
dataX.linear.__dict__
u_flutter = dataX.linear.dynamic_loads['flutter_results']['u_flutter']

print(u_flutter[0])

<class 'sharpy.structure.models.beam.Beam'>
<class 'sharpy.aero.models.aerogrid.Aerogrid'>
30.143130262718564


Things to look into the future: How flutter modes work and how they progress. Look at Hail documentation!! Need nice plots